In [1]:
import pandas as pd
import numpy as np

In [ ]:
main_df = pd.read_csv("main_df_csv.csv")
main_df.index = main_df.index+1
main_df

In [12]:
main_df.dropna(inplace = True)

In [84]:
main_df

,Volcano Name,Eruption Type,Max. VEI,Country,Volcanic_Region,Volcano_type,Summit,Year,Latitude_value,Longitude_value
0,Ambae,Confirmed Eruption,0,Vanuatu,Melanesia and Australia,13,1496,2021,-15.389,167.835
1,Pinatubo,Confirmed Eruption,0,Philippines,Philippines and SE Asia,14,1486,2021,15.130,120.350
2,Iliwerung,Confirmed Eruption,0,Indonesia,Indonesia,2,583,2021,-8.532,123.573
3,Taal,Confirmed Eruption,0,Philippines,Philippines and SE Asia,1,311,2021,14.002,120.993
4,Turrialba,Confirmed Eruption,0,Costa Rica,México and Central America,14,3340,2021,10.025,-83.767
...,...,...,...,...,...,...,...,...,...,...
9858,Craters of the Moon,Confirmed Eruption,0,United States,Canada and Western USA,11,2005,-60,43.420,-113.500
9859,Igwisi Hills,Confirmed Eruption,1,Tanzania,Africa and Red Sea,11,1146,-450,-4.889,31.933
9860,Quetrupillan,Confirmed Eruption,3,Chile,South America,14,2360,-658,-39.496,-71.722
9861,"Yojoa, Lago",Confirmed Eruption,0,Honduras,México and Central America,20,1060,-1073,14.964,-87.983


In [78]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9863 entries, 1 to 11219
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Volcano Name     9863 non-null   object 
 1   Eruption Type    9863 non-null   object 
 2   Max. VEI         9863 non-null   int8   
 3   Country          9863 non-null   object 
 4   Volcanic_Region  9863 non-null   object 
 5   Volcano_type     9863 non-null   object 
 6   Summit           9863 non-null   float64
 7   Year             9863 non-null   int32  
 8   Latitude_value   9863 non-null   float64
 9   Longitude_value  9863 non-null   float64
dtypes: float64(3), int32(1), int8(1), object(5)
memory usage: 741.7+ KB


In [60]:
main_df.describe(include="all")

,Volcano Name,Eruption Type,Max. VEI,Country,Volcanic_Region,Volcano_type,Summit,Year,Latitude_value,Longitude_value
count,9863,9863,9863.000000,9863,9863,9863,9863.000000,9863.000000,9863.000000,9863.000000
unique,856,1,NaN,80,19,20,NaN,NaN,NaN,NaN
top,Etna,Confirmed Eruption,NaN,Japan,"Japan, Taiwan, Marianas",Stratovolcano,NaN,NaN,NaN,NaN
freq,196,9863,NaN,1450,1514,6087,NaN,NaN,NaN,NaN
mean,NaN,NaN,1.565649,NaN,NaN,NaN,2166.932171,528.841022,16.826720,32.762365
std,NaN,NaN,1.317605,NaN,NaN,NaN,1370.101226,2538.586765,30.938184,115.044865
min,NaN,NaN,0.000000,NaN,NaN,NaN,-4200.000000,-9950.000000,-77.530000,-179.970000
25%,NaN,NaN,0.000000,NaN,NaN,NaN,1280.000000,290.000000,-6.520000,-77.370000
50%,NaN,NaN,2.000000,NaN,NaN,NaN,1940.000000,1841.000000,16.720000,55.708000
75%,NaN,NaN,2.000000,NaN,NaN,NaN,2948.000000,1950.000000,40.827000,139.282000


In [53]:
main_df.drop(main_df.loc[main_df["Eruption Type"] != 'Confirmed Eruption'].index, inplace=True)

In [ ]:
main_df['Max. VEI'] = main_df['Max. VEI'].replace(['--'],'0')
main_df['Max. VEI'] = pd.to_numeric(main_df['Max. VEI'],downcast= 'integer')#cast VEI from string to int

In [30]:
main_df[['Year', 'BCE']] = main_df['Start Date'].str.split(" ",n=1,expand=True)#split the data with spaces
main_df['Year'] = main_df['Year'].astype(int) #casting
main_df['BCE'].fillna(value = 'de', inplace = True)#filling nan vallues for the run
for index in range(11141):
    if main_df.iloc[index, 11] == "BCE":#if BCE turn the year to -1
        main_df.iloc[index, 10] *=  -1
main_df = main_df.drop(['BCE', 'Start Date'],axis=1) #drop the unecessary columns

In [45]:
main_df[['Latitude_value', 'direction']] = main_df['Latitude'].str.split("°",n=1,expand=True)#split the data with spaces
#main_df['symbol'].fillna(value = 'de', inplace = True)#filling nan vallues for the run
# N -> +
# S -> -
# E -> +
# W -> -
# in that order well be fixing the lat and long (also work in google maps)
main_df['Latitude_value'] = main_df['Latitude_value'].astype(float)
for index in range(11141):
    if main_df.iloc[index, 11] == "S":#if S turn the lat to -1
        main_df.iloc[index, 10] *=  -1
main_df = main_df.drop(['direction', 'Latitude'],axis=1) #drop the unecessary columns

In [49]:
main_df[['Longitude_value', 'direction']] = main_df['Longitude'].str.split("°",n=1,expand=True)#split the data with spaces
#main_df['symbol'].fillna(value = 'de', inplace = True)#filling nan vallues for the run
# N -> +
# S -> -
# E -> +
# W -> -
# in that order well be fixing the lat and long (also work in google maps)
main_df['Longitude_value'] = main_df['Longitude_value'].astype(float)
for index in range(11141):
    if main_df.iloc[index, 11] == "W":#if S turn the lat to -1
        main_df.iloc[index, 10] *=  -1
main_df = main_df.drop(['direction', 'Longitude'],axis=1) #drop the unecessary columns

In [37]:
#DO NOT DELETE
correlations=[]
arr=[]
cols1=main_df.columns
cols2=main_df.columns
i=0
for col1 in cols1:
    j=0
    for col2 in cols2:
        if col1==col2:
            break
        if main_df[col1].corr(main_df[col2])>=0.5 or main_df[col1].main_df[col1].corr(df[col2])>=0.5:
            arr.append(f"({main_df.columns[i]}, {main_df.columns[j]})")
        j+=1
    i+=1
print(arr)

In [74]:
main_df["Volcano_type"] = main_df["Volcano_type"].replace("\((\w+)\)", "",regex = True)#Regular Expressions - ze til!
volcano_type_dict = {"Caldera":1,"Complex":2,"Compound":3,"Cone":4,"Crater rows":5,"Explosion crater":6,
                     "Fissure vent":7,"Lava cone":8,"Lava dome":9,"Maar":10,"Pyroclastic cone":11,
                     "Pyroclastic shield":12,"Shield":13,"Stratovolcano":14,"Subglacial":15,"Submarine":16,
                     "Tuff cone":17,"Tuff ring":18,"Tuya":19,"Volcanic field":20,}
main_df["Volcano_type"] = main_df["Volcano_type"].replace(volcano_type_dict)
main_df["Volcano_type"] = main_df["Volcano_type"].astype(int) 

In [77]:
#####SAVE WORK END OF DAY ############

main_df.to_csv("updated df 16.01.22 with names.csv")

In [83]:
main_df = pd.read_csv("updated df 16.01.22 with names.csv")